In [3]:
import pygame
import random

# Define some constants for the game
GRID_SIZE = 4
TILE_SIZE = 100
WINDOW_SIZE = TILE_SIZE * GRID_SIZE
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46)
}

# utility functions

def get_direction_text(direction):
    return "up" if direction == 0 else "right" if direction == 1 else "down" if direction == 2 else "left"

pygame 2.5.2 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


In [4]:
class Game:
    def __init__(self, gui=True, grid=None):
        # the gui flag is used to determine whether to render the game or not (might be useful for testing)
        # Initialize the game
        if gui:
            self.gui = True
            pygame.init()
            self.window = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
            self.font = pygame.font.SysFont("Arial", 32)
        if not gui:
            self.gui = False
        self.score = 0
        if grid:
            self.grid = grid
        else:
            self.reset()

    def reset(self, grid=None):
        self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.add_random_tile()
        self.add_random_tile()
        # game over after 1 move for testing
        # self.grid = [
        #     [2, 4, 8, 16],
        #     [32, 64, 128, 256],
        #     [512, 1024, 2048, 256],
        #     [2, 4, 8, 16]
        # ]
        self.score = 0

    def add_random_tile(self):
        # Add a random tile to the grid, probability of adding a 2 is 90% and 4 is 10%
        empty_tiles = []
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    empty_tiles.append((i, j))
        if empty_tiles:
            i, j = random.choice(empty_tiles)
            self.grid[i][j] = 2 if random.random() < 0.9 else 4

    def render(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                # draw the tile
                pygame.draw.rect(self.window, TILE_COLORS[self.grid[i][j]], (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE))

                # add the text to the tile
                text = self.font.render(str(self.grid[i][j]) if self.grid[i][j] else "", True, (119, 110, 101))
                text_rect = text.get_rect()
                text_rect.center = (j * TILE_SIZE + TILE_SIZE / 2, i * TILE_SIZE + TILE_SIZE / 2)
                self.window.blit(text, text_rect)

                # add a border around the tiles
                pygame.draw.rect(self.window, (187, 173, 160), (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE), 5)

        pygame.display.update()

    def move(self, direction):
        # direction: 0 - up, 1 - right, 2 - down, 3 - left
        # slide the tiles in the given direction
        merged = [[False for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        initial_grid = [row[:] for row in self.grid]

        # Slide tiles as far as possible in the given direction, merging tiles of the same value once.
        if direction == 0:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i): # k checks all tiles between i and 0
                        if self.grid[k][j] == 0:
                            shift += 1
                    
                    if shift:
                        self.grid[i - shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i - shift - 1 >= 0 and self.grid[i - shift - 1][j] == self.grid[i - shift][j] and not merged[i - shift - 1][j] and not merged[i - shift][j]:
                        self.score += self.grid[i - shift][j] * 2
                        self.grid[i - shift - 1][j] *= 2
                        self.grid[i - shift][j] = 0
                        merged[i - shift - 1][j] = True

        elif direction == 1:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE - 1, -1, -1):
                    shift = 0
                    for k in range(j + 1, GRID_SIZE):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j + shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j + shift + 1 < GRID_SIZE and self.grid[i][j + shift + 1] == self.grid[i][j + shift] and not merged[i][j + shift + 1] and not merged[i][j + shift]:
                        self.score += self.grid[i][j + shift] * 2
                        self.grid[i][j + shift + 1] *= 2
                        self.grid[i][j + shift] = 0
                        merged[i][j + shift + 1] = True

        elif direction == 2:
            for i in range(GRID_SIZE - 1, -1, -1):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i + 1, GRID_SIZE):
                        if self.grid[k][j] == 0:
                            shift += 1

                    if shift:
                        self.grid[i + shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i + shift + 1 < GRID_SIZE and self.grid[i + shift + 1][j] == self.grid[i + shift][j] and not merged[i + shift + 1][j] and not merged[i + shift][j]:
                        self.score += self.grid[i + shift][j] * 2
                        self.grid[i + shift + 1][j] *= 2
                        self.grid[i + shift][j] = 0
                        merged[i + shift + 1][j] = True

        elif direction == 3:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(j):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j - shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j - shift - 1 >= 0 and self.grid[i][j - shift - 1] == self.grid[i][j - shift] and not merged[i][j - shift - 1] and not merged[i][j - shift]:
                        self.score += self.grid[i][j - shift] * 2
                        self.grid[i][j - shift - 1] *= 2
                        self.grid[i][j - shift] = 0
                        merged[i][j - shift - 1] = True

        if self.grid != initial_grid:
            self.add_random_tile()
            # print("Game State after moving " + get_direction_text(direction) + ":")
            # print(str(self))
            # print("Score: " + str(self.score))
            # print()
            if self.check_game_over():
                if self.gui:
                    pygame.time.delay(5000)
                    pygame.quit()
                return (1, self.grid)
            if self.gui:
                self.render()
            return (0, self.grid)
        else:
            return (2, self.grid)

    def check_game_over(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    return
                if i > 0 and self.grid[i - 1][j] == self.grid[i][j]:
                    return
                if i < GRID_SIZE - 1 and self.grid[i + 1][j] == self.grid[i][j]:
                    return
                if j > 0 and self.grid[i][j - 1] == self.grid[i][j]:
                    return
                if j < GRID_SIZE - 1 and self.grid[i][j + 1] == self.grid[i][j]:
                    return

        # overlay text on the screen
        if self.gui:
            text = self.font.render("Game Over!", True, (119, 110, 101))
            text_rect = text.get_rect()
            text_rect.center = (WINDOW_SIZE / 2, WINDOW_SIZE / 2)
            self.window.blit(text, text_rect)
            pygame.display.update()
        return True
        

    def __str__(self):
        return "\n".join([" ".join([str(self.grid[i][j]) for j in range(GRID_SIZE)]) for i in range(GRID_SIZE)])

### Run the cell below to play the game manually

In [ ]:
game = Game(grid=[
     [0, 0, 0, 0],
[2, 0, 0, 0],
[0, 2, 0, 0],
[0, 0, 0, 0],
])
game.render()

# close the game window when the user presses the close button
done = False
while not done:
    # game loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                game.reset()
                game.render()
            elif event.key == pygame.K_UP:
                if game.move(0)[0] == 1:
                    done = True
            elif event.key == pygame.K_RIGHT:
                if game.move(1)[0] == 1:
                    done = True
            elif event.key == pygame.K_DOWN:
                if game.move(2)[0] == 1:
                    done = True
            elif event.key == pygame.K_LEFT:
                if game.move(3)[0] == 1:
                    done = True
        else:
            continue

### Random moves simulation

In [69]:
game = Game()
game.render()

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
    pygame.time.delay(200)
    if game.move(random.randint(0, 3))[0] == 1:
        done = True
        

In [70]:
# stats for 1000 random games
Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0
for i in range(1000):
    game = Game(gui=False)
    while not game.check_game_over():
        game.move(random.randint(0, 3))
    Sum += game.score
    Max = max(Max, game.score)
    if Max == game.score:
        Max_game = game
    if any(2048 in row for row in game.grid):
        _2048prob += 1
    if any(1024 in row for row in game.grid):
        _1024prob += 1
    if any(512 in row for row in game.grid):
        _512prob += 1
    if any(256 in row for row in game.grid):
        _256prob += 1
    if any(128 in row for row in game.grid):
        _128prob += 1
    if any(64 in row for row in game.grid):
        _64prob += 1
    if any(32 in row for row in game.grid):
        _32prob += 1
    if any(16 in row for row in game.grid):
        _16prob += 1
    if any(8 in row for row in game.grid):
        _8prob += 1
    if any(4 in row for row in game.grid):
        _4prob += 1
    if any(2 in row for row in game.grid):
        _2prob += 1

print("Average score: " + str(Sum / 1000))
print("Maximum score: " + str(Max))
print("Maximum score grid: ")
print(str(Max_game))
print("2 probability: " + str(_2prob / 1000))
print("4 probability: " + str(_4prob / 1000))
print("8 probability: " + str(_8prob / 1000))
print("16 probability: " + str(_16prob / 1000))
print("32 probability: " + str(_32prob / 1000))
print("64 probability: " + str(_64prob / 1000))
print("128 probability: " + str(_128prob / 1000))
print("256 probability: " + str(_256prob / 1000))
print("512 probability: " + str(_512prob / 1000))
print("1024 probability: " + str(_1024prob / 1000))
print("2048 probability: " + str(_2048prob / 1000))
print("4096 probability: " + str(_4096prob / 1000))

Average score: 1087.408
Maximum score: 3084
Maximum score grid: 
4 8 2 16
64 256 32 2
4 8 128 8
2 16 4 2
2 probability: 1.0
4 probability: 1.0
8 probability: 0.987
16 probability: 0.948
32 probability: 0.855
64 probability: 0.704
128 probability: 0.482
256 probability: 0.085
512 probability: 0.0
1024 probability: 0.0
2048 probability: 0.0
4096 probability: 0.0


## Uninformed search on the state space

### BFS

In [71]:
game = Game(gui=False)

def bfs(game):
    # breadth first search to find the best move
    # returns a tuple (best_score, moves)
    # moves: list of moves taken to reach the best score
    # game: the game object
    queue = []
    visited = set()
    queue.append((game.grid, 0))
    visited.add(str(game.grid))
    best_score = 0
    moves = [] # list of moves taken to reach the best score
    while queue:
        grid, score = queue.pop(0)
        if score > best_score:
            best_score = score
        for i in range(4):
            game.grid = grid
            game.move(i)
            if str(game.grid) not in visited:
                queue.append((game.grid, score + game.score))
                moves.append(i)
                visited.add(str(game.grid))
    return (best_score, moves)

print("Game State:\n", str(game))
(game.score, moves) = bfs(game)
print("Moves: " + str([get_direction_text(move) for move in moves]))
print("Final State:\n", str(game))
print("Score: " + str(game.score))
print("Max Tile: " + str(max(max(row) for row in game.grid)))    

Game State:
 0 0 2 0
0 0 0 2
0 0 0 0
0 0 0 0
Moves: ['up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'down', 'left', 'up', 'right', 'd

In [73]:
# stats for 1000 random games
Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0

for i in range(1000):
    game = Game(gui=False)
    (game.score, moves) = bfs(game)
    Sum += game.score
    Max = max(Max, game.score)
    if Max == game.score:
        Max_game = game
    if any(2048 in row for row in game.grid):
        _2048prob += 1
    if any(1024 in row for row in game.grid):
        _1024prob += 1
    if any(512 in row for row in game.grid):
        _512prob += 1
    if any(256 in row for row in game.grid):
        _256prob += 1
    if any(128 in row for row in game.grid):
        _128prob += 1
    if any(64 in row for row in game.grid):
        _64prob += 1
    if any(32 in row for row in game.grid):
        _32prob += 1
    if any(16 in row for row in game.grid):
        _16prob += 1
    if any(8 in row for row in game.grid):
        _8prob += 1
    if any(4 in row for row in game.grid):
        _4prob += 1
    if any(2 in row for row in game.grid):
        _2prob += 1

print("Average score: " + str(Sum / 1000))
print("Maximum score: " + str(Max))
print("Maximum score grid: ")
print(str(Max_game))
print("2 probability: " + str(_2prob / 1000))
print("4 probability: " + str(_4prob / 1000))
print("8 probability: " + str(_8prob / 1000))
print("16 probability: " + str(_16prob / 1000))
print("32 probability: " + str(_32prob / 1000))
print("64 probability: " + str(_64prob / 1000))
print("128 probability: " + str(_128prob / 1000))
print("256 probability: " + str(_256prob / 1000))
print("512 probability: " + str(_512prob / 1000))
print("1024 probability: " + str(_1024prob / 1000))
print("2048 probability: " + str(_2048prob / 1000))
print("4096 probability: " + str(_4096prob / 1000))


Average score: 2687.344
Maximum score: 7728
Maximum score grid: 
4 64 8 2
32 8 256 16
16 512 2 32
4 8 4 2
2 probability: 1.0
4 probability: 0.998
8 probability: 0.99
16 probability: 0.962
32 probability: 0.891
64 probability: 0.8
128 probability: 0.713
256 probability: 0.501
512 probability: 0.053
1024 probability: 0.0
2048 probability: 0.0
4096 probability: 0.0


### DFS

In [ ]:
game = Game(gui=False)

def dfs(game):
    

print("Game State:\n", str(game))
(game.score, moves) = dfs(game)
print("Moves: " + str([get_direction_text(move) for move in moves]))
print("Final State:\n", str(game))
print("Score: " + str(game.score))
print("Max Tile: " + str(max(max(row) for row in game.grid)))




### Informed search on the state space

Before performing informed search, we need to define a heuristic function.
Some helpful functions are provided below.


In [1]:
def largest_tile(game):
    # returns the largest tile in the grid
    return max(max(row) for row in game.grid)

def empty_tiles(game):
    # returns the number of empty tiles in the grid, more empty tiles provide more flexibility
    return sum(1 for row in game.grid for tile in row if tile == 0)

def monotonicity(game):
    # returns the sum of the differences between adjacent tiles in each row and column
    # monotonicity is good because it means that the tiles are either increasing or decreasing in value
    # which means that they can be merged in the future
    # this is a heuristic that favors grids where the tiles are increasing or decreasing in value
    # in one direction
    # for example, a grid like this would have a high monotonicity score:
    # 2 4 8 16
    # 4 8 16 32
    # 8 16 32 64
    # 16 32 64 128
    # because the tiles are increasing in value from left to right and top to bottom
    # a grid like this would have a low monotonicity score:
    # 2 4 8 16
    # 4 2 16 32
    # 8 16 2 64
    # 16 32 64 2
    
    row_score = 0
    col_score = 0

    for row in game.grid:
        inc = sum(1 for i in range(1, len(row)) if row[i] - row[i - 1] > 0)
        dec = sum(1 for i in range(1, len(row)) if row[i] - row[i - 1] < 0)
        row_score += max(inc, dec)
        
    for col in zip(*game.grid):
        inc = sum(1 for i in range(1, len(col)) if col[i] - col[i - 1] > 0)
        dec = sum(1 for i in range(1, len(col)) if col[i] - col[i - 1] < 0)
        col_score += max(inc, dec)

    return row_score + col_score

def largest_tile_in_corner(game):
    largest_tile = max(max(row) for row in game.grid)

    if game.grid[0][0] == largest_tile or game.grid[0][GRID_SIZE - 1] == largest_tile or game.grid[GRID_SIZE - 1][0] == largest_tile or game.grid[GRID_SIZE - 1][GRID_SIZE - 1] == largest_tile:
        return 1
    else:
        return 0
    
def smoothness(game):
    # returns the sum of the differences between adjacent tiles in each row and column
    # smoothness is good because it means that the tiles are close in value
    # which means that they can be merged in the future
    # this is a heuristic that favors grids where the tiles are close in value
    # for example, a grid like this would have a high smoothness score:
    # 2 4 8 16
    # 4 8 16 32
    # 8 16 32 64
    # 16 32 64 128
    # because the tiles are close in value to each other
    # a grid like this would have a low smoothness score:
    # 2 256 8 16
    # 512 128 32 64
    # 8 16 32 64
    # 16 32 64 128


    
    row_score = 0
    col_score = 0

    for row in game.grid:
        row_score += sum(abs(row[i] - row[i - 1]) for i in range(1, len(row)))
        
    for col in zip(*game.grid):
        col_score += sum(abs(col[i] - col[i - 1]) for i in range(1, len(col)))

    # lower smoothness score is better
    return row_score + col_score




#### A* search with different heuristic functions

In [5]:
def heuristic(game):
    # returns a heuristic score for the game
    # the heuristic score is a weighted sum of the following:
    # 1. the largest tile in the grid
    # 2. the number of empty tiles in the grid
    # 3. the monotonicity of the grid
    # 4. the smoothness of the grid
    # 5. the largest tile in the corner
    # the weights are:
    # 1. 100
    # 2. 10
    # 3. 1
    # 4. 1
    # 5. 1000
    # the weights are determined by trial and error
    # the weights can be changed to improve the heuristic
    return 100 * largest_tile(game) + 10 * empty_tiles(game) + 1 * monotonicity(game) + 1 * smoothness(game) + 1000 * largest_tile_in_corner(game)

def a_star(game, moves=[]):
    # a* search to find the best move
    # returns a tuple (best_score, moves)
    # moves: list of moves taken to reach the best score
    # game: the game object
    possible_states = []
    for i in range(4):
        new_game = Game(grid=game.grid)
        if new_game.move(i)[0] == 0:
            possible_states.append((new_game.grid, i))
    possible_states = sorted(possible_states, key=lambda x: heuristic(Game(grid=x)) + len(moves))

    if not possible_states:
        return (game.score, moves)
    else:
        return a_star(Game(grid=possible_states[0]), moves + get_direction_text(possible_states[0][1]))

game = Game(gui=False)
print("Game State:\n", str(game))
(game.score, moves) = a_star(game)
print("Moves: " + str(moves))
print("Final State:\n", str(game))
print("Score: " + str(game.score))

    



Game State:
 0 2 0 0
0 0 0 0
0 0 0 0
0 2 0 0


TypeError: 'int' object is not iterable